## Metodo con Selenium

In [ ]:
# En caso de que no tengamos gdown o selenium instalado, procedemos a descargarlo mediante pip
!pip install gdown
!pip install selenium

# Primeramente descargamos el archivo ChromeDriver desde Google Drive
!gdown --id 1w78mIreuYtixvsmVcgdKLP_mAy4KxMzv

# Luego cambiamos el permiso para executar el chromedriver
!chmod +x /content/chromedriver
# Luego movemos el archivo chromedriver al directorio adecuado
!mv /content/chromedriver /usr/local/bin/chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pprint
import traceback  # Importamos traceback para obtener detalles completos del error


# Configuramos opciones para Chrome en modo headless
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializamos el controlador de Chrome con las opciones configuradas
driver = webdriver.Chrome(options=chrome_options)

# Función para realizar la búsqueda de un producto y obtener las URLs de los primeros 10 resultados
def obtener_urls_productos(busqueda, cantidad=10):
    # Construimos la URL de búsqueda con el término proporcionado
    url_busqueda = f"https://listado.mercadolibre.com.co/{busqueda}"
    driver.get(url_busqueda)
    try:
        # Esperamos hasta que los elementos de la lista de resultados estén presentes
        productos = WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.ui-search-layout__item a'))
        )

        # Obtenemos las URLs de los primeros 10 productos
        urls = [producto.get_attribute('href') for producto in productos[:cantidad]]
        return urls
    except Exception as e:
        print(f"Error al obtener las URLs de los productos: {e}")
        traceback.print_exc()  # Imprime el traceback completo del error
        return []

# Función para obtener los detalles de un producto
def obtener_detalles_producto(url_producto):
    data = {}
    try:
        driver.get(url_producto)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ui-pdp-title')))

        # Extraemos los detalles del producto usando bloques try-except
        try:
            data['Categoria'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-breadcrumb').text.strip()
        except:
            data['Categoria'] = None

        try:
            data['Titulo'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-title').text.strip()
        except:
            data['Titulo'] = None

        try:
            data['Precio'] = driver.find_element(By.CSS_SELECTOR, 'div.ui-pdp-price__second-line span.andes-money-amount__fraction').text.strip()
        except:
            data['Precio'] = None

        try:
            discount = driver.find_element(By.CSS_SELECTOR, 's[role="img"][aria-label^="Antes:"]')
            data['Descuento'] = True if discount else False
        except:
            data['Descuento'] = False

        try:
            seller_button = driver.find_element(By.CLASS_NAME, 'ui-pdp-seller__link-trigger-button')
            seller = seller_button.find_elements(By.TAG_NAME, 'span')[1].text.strip()
            data['Vendedor'] = seller
        except:
            data['Vendedor'] = None

        try:
            data['Calificacion promedio'] = driver.find_element(By.CSS_SELECTOR, 'span.ui-pdp-review__rating[aria-hidden="true"]').text.strip()
        except:
            data['Calificacion promedio'] = None

        try:
            data['Cantidad de Calificaciones'] = driver.find_element(By.CSS_SELECTOR, 'span.ui-pdp-review__amount[aria-hidden="true"]').text.strip()
        except:
            data['Cantidad de Calificaciones'] = None

        try:
            warranty = driver.find_element(By.XPATH, "//p[contains(., 'garantía de fábrica.')]")
            data['Garantia'] = warranty.text.strip() if 'garantía de fábrica.' in warranty.text.strip() else None
        except:
            data['Garantia'] = None

        try:
            data['Descripcion'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-description__content').text.strip()
        except:
            data['Descripcion'] = None

        try:
            data['Stock'] = driver.find_element(By.CLASS_NAME, 'ui-pdp-stock-information__title').text.strip()
        except:
            data['Stock'] = None

        try:
            data['Cantidad de Opiniones'] = driver.find_element(By.CLASS_NAME, 'total-opinion').text.strip()
        except:
            data['Cantidad de Opiniones'] = None

        try:
            data['Numero de Publicacion'] = driver.find_element(By.CSS_SELECTOR, 'span.ui-pdp-color--BLACK.ui-pdp-family--SEMIBOLD').text.strip()
        except:
            data['Numero de Publicacion'] = None

        # Agregamos la URL del producto al diccionario
        data['URL del Producto'] = url_producto

    except Exception as e:
        print(f"Error al extraer los datos del producto: {e}")
    return data

# Realizamos la búsqueda y obtenemos los detalles de los primeros 10 productos
busqueda = 'portatil'
urls_productos = obtener_urls_productos(busqueda)

# Lista para almacenar los detalles de cada producto
detalles_productos = []

# Iteramos sobre cada URL y obtenemos los detalles del producto
for url in urls_productos:
    detalles_producto = obtener_detalles_producto(url)
    detalles_productos.append(detalles_producto)

# Finalmente, imprimimos los detalles de todos los productos de manera legible
pprint.pprint(detalles_productos)

# Cerramos el navegador al final
driver.quit()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 30.9 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1w78mIreuYtixvsmVcgdKLP_mAy4KxMzv
To: /content/chromedriver
100% 17.3M/17.3M [00:00<00:00, 33.1MB/s]
Error al extraer los datos del producto: Message: 
Stacktrace:
#0 0x55831ab9210a <unknown>
#1 0x55831a6ad460 <unknown>
#2 0x55831a6f9f96 <unknown>
#3 0x55831a6fa221 <unknown>
#4 0x55831a73f5d4 <unknown>
#5 0x55831a71e03d <unknown>
#6 0x55831a73caf6 <unknown>
#7 0x55831a71ddb3 <unknown>
#8 0x55831a6ecc10 <unknown>
#9 0x55831a6edbee <unknown>
#10 0x55831ab5e39b <unknown>
#11 0x55831ab62338 <unknown>
#12 0x55831ab4bebc <unknown>
#13 0x55831ab62eb7 <unknown>
#14 0x558